In [553]:
from proxmin import nmf
from proxmin.utils import Traceback
from scipy.optimize import linear_sum_assignment
from scipy.stats import binned_statistic
import numpy as np
import matplotlib.pyplot as plt
import time
from proxmin import operators as po
from functools import partial

In [554]:
data = np.load("hsc_stacked.npy")

In [555]:
redshifts = data['z_cl']

# Bin the cluster data by redshift
clusters1 = data[(redshifts > 0.1) & (redshifts < 0.2)]
clusters2 = data[(redshifts > 0.2) & (redshifts < 0.35)]
clusters3 = data[(redshifts > 0.35) & (redshifts < 0.5)]
clusters4 = data[(redshifts > 0.5) & (redshifts < 0.75)]
clusters5 = data[(redshifts > 0.75) & (redshifts < 1.2)]

In [556]:
num_radial_bins = 15

In [557]:
# Compute the average colors (and their variances) for a given redshift bin

def get_clusters_data(clusters):
    
    clusters = clusters[np.where(clusters['R'] > 0.1)]
    radii = np.array(clusters['R'])
    
    # Arrays containing magnitude data in four different filters
    g = np.array(clusters['gmag_forced_cmodel'])
    r = np.array(clusters['rmag_forced_cmodel'])
    i = np.array(clusters['imag_forced_cmodel'])
    z = np.array(clusters['zmag_forced_cmodel'])
    
    g_pruned = g[np.isfinite(g) & np.isfinite(r) & np.isfinite(i) & np.isfinite(z)]
    r_pruned = r[np.isfinite(g) & np.isfinite(r) & np.isfinite(i) & np.isfinite(z)]
    i_pruned = i[np.isfinite(g) & np.isfinite(r) & np.isfinite(i) & np.isfinite(z)]
    z_pruned = z[np.isfinite(g) & np.isfinite(r) & np.isfinite(i) & np.isfinite(z)]

    radii_pruned = radii[np.isfinite(g) & np.isfinite(r) & np.isfinite(i) & np.isfinite(z)]
    radial_bins = np.logspace(np.log10(np.amin(radii_pruned)), np.log10(0.9*np.amax(radii_pruned)), num_radial_bins + 1)
    widths = np.diff(radial_bins)
    annular_areas = np.pi * np.diff(radial_bins ** 2.)

    
    # For each color g-r, r-i, and i-z: first, throw out data points that have magnitudes of "nan" or "inf" (pruning the
    # arrays of radii and magnitudes accordingly), then use scipy.binned_statistic to sort the data into radial bins and
    # compute the number of objects and the sum and variance (noise^2) of the color values in those bins

    
    gr = g_pruned - r_pruned
    counts = binned_statistic(radii_pruned, gr, 'count', radial_bins)[0]
    sum_gr = binned_statistic(radii_pruned, gr, 'sum', radial_bins)[0]
    var_gr = binned_statistic(radii_pruned, gr, np.var, radial_bins)[0]
    var_gr *= counts

    ri = r_pruned - i_pruned
    sum_ri = binned_statistic(radii_pruned, ri, 'sum', radial_bins)[0]
    var_ri = binned_statistic(radii_pruned, ri, np.var, radial_bins)[0]
    var_ri *= counts

    iz = i_pruned - z_pruned
    sum_iz = binned_statistic(radii_pruned, iz, 'sum', radial_bins)[0]
    var_iz = binned_statistic(radii_pruned, iz, np.var, radial_bins)[0]
    var_iz *= counts
    
    return (radial_bins, widths, annular_areas, counts, (sum_gr, var_gr), (sum_ri, var_ri), (sum_iz, var_iz))

In [596]:
# Use NMF to unmix the color profiles generated by plot_profiles and avg_colors
# The arguments of "unmix" are 2-tuples of the form (mean_color, var_color). For example, gr is really (mean_gr, var_gr)

def prox_field(S, step):
    S[0, :] = 1
    S[1:, -1] = 0
    S[1:, :] /= S[1:,:].max(axis=1)[:,None]
    return S

def prox_ndens(X, step):
    X[0, :] = np.minimum(0, X[0, :])

def unmix(counts, gr, ri, iz, annular_areas):
    n = num_radial_bins     # component resolution
    k = 2                   # number of components
    b = 4                   # number of observations (b=4 for counts, g-r, r-i, and i-z)

    # Data matrix to be unmixed
    Y = np.array([counts, gr[0], ri[0], iz[0]]) / annular_areas
    
    # if noise is variable, specify variance matrix of the same shape as Y
    W = 1. / (np.array([counts, gr[1], ri[1], iz[1]]) / annular_areas**2)

    # initialize and run NMF
    A = np.random.uniform(size = (b, k))
    S = np.random.uniform(size = (k, n))
    
    pA = prox_ndens
    pS = po.AlternatingProjections([prox_field, po.prox_plus])
    
    nmf(Y, A, S, W=W, prox_A=pA, prox_S=pS, e_rel=1e-6, e_abs=0)
    
    return A, S

In [597]:
def plot_profiles(clusters_data, z_bin_num, area_normalization=True):
    

    radial_bins = clusters_data[0]
    bin_midpoints = (radial_bins[1:] * radial_bins[:-1]) ** 0.5
    widths = clusters_data[1]
    annular_areas = clusters_data[2]
    counts = clusters_data[3]
    
    # gr, ri, iz, are 2-tuples of the form (sum_color, var_color)
    gr = clusters_data[4]
    ri = clusters_data[5]
    iz = clusters_data[6]
    
    A, S = unmix(counts, gr, ri, iz, annular_areas)
    print("A:\n", A, "\n")
    print("S:\n", S, "\n")
    
    z_bins = ("1 (0.1 < z < 0.2)", "2 (0.2 < z < 0.35)", "3 (0.35 < z < 0.5)", "4 (0.5 < z < 0.75)", 
             "5 (0.75 < z < 1.2)")
    
    # Plot S_k (the relative abundance of the kth component) vs. radius for each k
    
    for k in range(S.shape[0]):
        plt.plot(bin_midpoints, S[k], label = "Component %s" % (k+1))
    plt.xscale('log')
    plt.title("Relative Component Abundance vs. Radius for Redshift Bin %s" % z_bins[z_bin_num - 1])
    plt.xlabel("Radius from cluster center [Mpc]")
    plt.ylabel("Relative abundance")
    plt.legend()
    plt.show()
    

    Y_remixed = np.dot(A, S)
    
    # Plot the reconstructed color profiles following NMF 
    # (that is, plot Y_remixed = A x S, where A and S are the results of factorizing the initial data matrix Y)
    
    if(area_normalization):
        
        plt.bar(radial_bins[:-1], counts/annular_areas, width=widths, fill=False, align="edge", 
                yerr=np.sqrt(counts/annular_areas**2.), label="Data")
        plt.plot(bin_midpoints, Y_remixed[0], label="Predicted Profile")
        for k in range(S.shape[0]):
            plt.plot(bin_midpoints, S[k], label = "Component %s" % (k+1))
        plt.xscale('log')
        plt.title("Galaxy Number Density vs. Radius for Redshift Bin %s" % z_bins[z_bin_num - 1])
        plt.xlabel("Radius from cluster center [Mpc]")
        plt.ylabel(r"Number Density $[\frac{counts}{Mpc^2}]$")
        plt.legend()
        plt.show()
        
        plt.bar(radial_bins[:-1], gr[0]/annular_areas, width=widths, fill=False, align="edge", 
                yerr=np.sqrt(gr[1]/annular_areas**2.), label="Data")
        plt.plot(bin_midpoints, Y_remixed[1], label="Predicted Profile")
        plt.xscale('log')
        plt.title("g-r Surface Brightness vs. Radius for Redshift Bin %s" % z_bins[z_bin_num - 1])
        plt.xlabel("Radius from cluster center [Mpc]")
        plt.ylabel(r"$\frac{g-r}{Area} [\frac{magnitudes}{Mpc^2}]$")
        plt.legend()
        plt.show()

        plt.bar(radial_bins[:-1], ri[0]/annular_areas, width=widths, fill=False, align="edge", 
                yerr=np.sqrt(ri[1]/annular_areas**2.), label="Data")
        plt.plot(bin_midpoints, Y_remixed[2], label="Predicted Profile")
        plt.xscale('log')
        plt.title("r-i Surface Brightness vs. Radius for Redshift Bin %s" % z_bins[z_bin_num - 1])
        plt.xlabel("Radius from cluster center [Mpc]")
        plt.ylabel(r"$\frac{r-i}{Area} [\frac{magnitudes}{Mpc^2}]$")
        plt.legend()
        plt.show()

        plt.bar(radial_bins[:-1], iz[0]/annular_areas, width=widths, fill=False, align="edge", 
                yerr=np.sqrt(iz[1]/annular_areas**2.), label="Data")
        plt.plot(bin_midpoints, Y_remixed[3], label="Predicted Profile")
        plt.xscale('log')
        plt.title("i-z Surface Brightness vs. Radius for Redshift Bin %s" % z_bins[z_bin_num - 1])
        plt.xlabel("Radius from cluster center [Mpc]")
        plt.ylabel(r"$\frac{i-z}{Area} [\frac{magnitudes}{Mpc^2}]$")
        plt.legend()
        plt.show()

        
    else:
        plt.bar(radial_bins[:-1], gr[0], width=widths, align="edge", yerr=np.sqrt(gr[1]), label="Data")
        plt.plot(bin_midpoints, Y_remixed[1], label="Predicted Profile")
        plt.xscale('log')
        plt.title("Average g-r Color vs. Radius for Redshift Bin %s" % z_bins[z_bin_num - 1])
        plt.xlabel("Radius from cluster center [Mpc]")
        plt.ylabel ("Average g-r [magnitudes]")
        plt.legend()
        plt.show()

        plt.bar(radial_bins[:-1], ri[0], width=widths, align="edge", yerr=np.sqrt(ri[1]), label="Data")
        plt.plot(bin_midpoints, Y_remixed[2], label="Predicted Profile")
        plt.xscale('log')
        plt.title("Average r-i Color vs. Radius for Redshift Bin %s" % z_bins[z_bin_num - 1])
        plt.xlabel("Radius from cluster center [Mpc]")
        plt.ylabel ("Average r-i [magnitudes]")
        plt.legend()
        plt.show()

        plt.bar(radial_bins[:-1], iz[0], width=widths, align="edge", yerr=np.sqrt(iz[1]), label="Data")
        plt.plot(bin_midpoints, Y_remixed[3], label="Predicted Profile")
        plt.xscale('log')
        plt.title("Average i-z Color vs. Radius for Redshift Bin %s" % z_bins[z_bin_num - 1])
        plt.xlabel("Radius from cluster center [Mpc]")
        plt.ylabel ("Average i-z [magnitudes]")
        plt.legend()
        plt.show()

In [598]:
#Plot bar graphs for the clusters1 data

clusters1_data = get_clusters_data(clusters1)

In [599]:
#Plot bar graphs for the post-NMF clusters1 profiles

plot_profiles(clusters1_data, 1, area_normalization=True)

ArpackNoConvergence: ARPACK error -1: No convergence (301 iterations, 0/1 eigenvectors converged) [ARPACK error -14: DNAUPD  did not find any eigenvalues to sufficient accuracy.]

In [ ]:
#Plot bar graphs for the clusters2 data

clusters2_data = get_clusters_data(clusters2)

In [ ]:
#Plot bar graphs for the post-NMF clusters2 profiles

plot_profiles(clusters2_data, 2, area_normalization=True)

In [ ]:
#Plot bar graphs for the clusters3 data

clusters3_data = get_clusters_data(clusters3)

In [ ]:
#Plot bar graphs for the post-NMF clusters3 profiles

plot_profiles(clusters3_data, 3, area_normalization=True)

In [ ]:
#Plot bar graphs for the clusters4 data

clusters4_data = get_clusters_data(clusters4)

In [ ]:
#Plot bar graphs for the post-NMF clusters4 profiles

plot_profiles(clusters4_data, 4, area_normalization=True)

In [ ]:
#Plot bar graphs for the clusters5 data

clusters5_data = get_clusters_data(clusters5)

In [ ]:
#Plot bar graphs for the post-NMF clusters5 profiles

plot_profiles(clusters5_data, 5, area_normalization=True)